# Pacotes / Bibliotecas

In [ ]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Ambiente

In [ ]:
enviroment_name = 'CartPole-v0'
env = gym.make(enviroment_name )

In [ ]:
episodes = 5

for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {}  Score: {}'.format(episode, score))
        
env.close()

# Treinando o modelo AR

In [ ]:
parent_dir = os.getcwd()
path = os.path.join(parent_dir,'Training')
try:
    os.mkdir(path)
    os.mkdir(os.path.join(path,'Logs'))
    os.mkdir(os.path.join(path,'Saved Models'))
except OSError as e:
    print(e)

In [ ]:
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(enviroment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

In [ ]:
model.learn(total_timesteps = 20000)

# Salvando e recarregando o modelo

In [ ]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_Path)

# Avaliação

In [ ]:
evaluate_policy(model, env, n_eval_episodes = 10, render = True)

In [ ]:
env.close()

# Testando modelo

In [ ]:
episodes = 5

for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episode: {}  Score: {}'.format(episode, score))
        
env.close()

# Visualizando Logs com Tensorboard

In [ ]:
training_path_log = os.path.join(log_path, 'PPO_2')

In [ ]:
training_path_log

In [ ]:
!tensorboard --logdir={training_path_log}

# Callback

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join('Training', 'Saved Models')

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold = 200, verbose = 1)
eval_callback = EvalCallback(env,
                            callback_on_new_best = stop_callback,
                             eval_freq = 10000,
                             best_model_save_path = save_path,
                             verbose = 1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

In [ ]:
model.learn(total_timesteps = 20000, callback = eval_callback)

# Mudando políticas

In [ ]:
net_arch = [dict(pi = [128, 128, 128, 128], vf = [128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path, policy_kwargs = {'net_arch':net_arch})

In [ ]:
model.learn(total_timesteps = 20000, callback = eval_callback)

# Mudando algoritmos

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

In [ ]:
model.learn(total_timesteps = 20000)